<a href="https://colab.research.google.com/github/YoonJaeJoung/detect_mask_AI/blob/main/face_mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##2. Read XML, crop image

download data from:
https://www.kaggle.com/datasets/andrewmvd/face-mask-detection

and save to '/content/drive/MyDrive/dataset' (create dataset folder and upload the data to it)

In [33]:
import os

os.makedirs('/content/drive/MyDrive/dataset/cropped/mask_weared_incorrect')
os.mkdir('/content/drive/MyDrive/dataset/cropped/with_mask')
os.mkdir('/content/drive/MyDrive/dataset/cropped/without_mask')

In [34]:
import xml.etree.ElementTree as ET

# read_xml function
def objects(name, xml_path):
    
    # parse xml file
    tree = ET.parse(xml_path + name + ".xml") 
    root = tree.getroot() # get root object

    bbox_coordinates = []
    for member in root.findall('object'):
        class_name = member[0].text # class name
            
        # bbox coordinates
        xmin = int(member[5][0].text)
        ymin = int(member[5][1].text)
        xmax = int(member[5][2].text)
        ymax = int(member[5][3].text)
        
        # store data in list
        bbox_coordinates.append([class_name, xmin, ymin, xmax, ymax])

    return bbox_coordinates

In [35]:
from PIL import Image

path = "/content/drive/MyDrive/dataset/"

# extrat data from the xml file
def extract(count, xml_path = path + 'annotations/'):
  data = []

  for num in range(count + 1):
    name = 'maksssksksss' + str(num)
    data.append(objects(name, xml_path))
    
  return data

# crop image
def img_crop(name, area): # area: [xmin, ymin, xmax, ymax]
  img = Image.open(name)
  
  xmin, ymin, xmax, ymax = area[0], area[1], area[2], area[3]
  width, height = xmax - xmin, ymax - ymin
  if width > height:
    ymin = ymin - abs(width-height) / 2
    ymax = ymax + abs(width-height) / 2
  elif width < height:
    xmin = xmin - abs(width-height) / 2
    xmax = xmax + abs(width-height) / 2
  cropped_img = img.crop((xmin, ymin, xmax, ymax))
  
  return cropped_img

# crop and save image
def crop_and_save(data, img_path = path + 'images/'):
  count = 0
  for num in range(len(data)):
    name = img_path + 'maksssksksss' + str(num) + '.png'
    
    for i in range(len(data[num])):
      file = data[num][i]
      print(count, num, file)
      # without_mask : 2, with_mask: 1, mask_weared_incorrect: 0
      save_path = path + 'cropped/' + file[0] + '/'
      img_crop(name, file[1:]).save(save_path+str(count)+'.png', 'PNG')
      count += 1
      
# number of xml-img file sets
num = 852

crop_and_save(extract(num))

0 0 ['without_mask', 79, 105, 109, 142]
1 0 ['with_mask', 185, 100, 226, 144]
2 0 ['without_mask', 325, 90, 360, 141]
3 1 ['with_mask', 321, 34, 354, 69]
4 1 ['with_mask', 224, 38, 261, 73]
5 1 ['with_mask', 299, 58, 315, 81]
6 1 ['with_mask', 143, 74, 174, 115]
7 1 ['with_mask', 74, 69, 95, 99]
8 1 ['with_mask', 191, 67, 221, 93]
9 1 ['with_mask', 21, 73, 44, 93]
10 1 ['with_mask', 369, 70, 398, 99]
11 1 ['without_mask', 83, 56, 111, 89]
12 2 ['with_mask', 68, 42, 105, 69]
13 2 ['with_mask', 154, 47, 178, 74]
14 2 ['with_mask', 238, 34, 262, 69]
15 2 ['mask_weared_incorrect', 333, 31, 366, 65]
16 3 ['with_mask', 52, 53, 73, 76]
17 3 ['with_mask', 72, 53, 92, 75]
18 3 ['with_mask', 112, 51, 120, 68]
19 3 ['with_mask', 155, 60, 177, 83]
20 3 ['with_mask', 189, 59, 210, 80]
21 3 ['with_mask', 235, 57, 257, 78]
22 3 ['with_mask', 289, 60, 309, 83]
23 3 ['with_mask', 313, 68, 333, 90]
24 3 ['with_mask', 351, 35, 364, 59]
25 4 ['with_mask', 70, 185, 176, 321]
26 5 ['with_mask', 118, 54, 161

##3. Import Module (Need for 3~)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split

##4. Train Model

In [38]:
# load image data
path = '/content/drive/MyDrive/dataset/cropped'
transform = transforms.Compose([transforms.Resize(130),
                                transforms.CenterCrop(128),
                                transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor()
                                ])
# 4000 images

In [39]:
# create dataset
# 0: mask_weared_incorrect, 1: with_mask, 2: without_mask
dataset = datasets.ImageFolder(path, transform=transform)
# dataset: set of (tensor, label)
dataset_size = len(dataset)

# split train / test data
train_size = int(dataset_size * 0.8) # 3200
test_size = dataset_size - train_size # 800
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True) # 100 batch
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
# dataloader: set of [imgs_tensor, labels_tensor], each include 32 (single batch)

In [ ]:
# show each image loaded
'''
print(len(train_dataloader))
for images, labels in train_dataloader:
  for i in range(len(images)):
    
    plt.imshow(  images[i].permute(1, 2, 0)  )
    if labels[i].item() == 1:
      plt.show()
      
print(len(test_dataloader))
for images, labels in test_dataloader:
  print(images.shape)
  print(labels.shape)
  
'''

In [40]:
# CNN 모델 클래스를 정의합니다.
class CNN(nn.Module):
      # define model structure
    def __init__(self):
        # call parent class(nn.Module)'s constructor
        super().__init__()
        
        # first hidden layer
        self.hidden_layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3), padding=1), # size: 128, 64 layers
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)), # size: 64, 64 layers
            nn.Dropout(0.5)
        )
        # second hidden layer
        self.hidden_layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3), padding=1), # size: 64, 128 layers
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)), # size: 32, 128 layers
            nn.Dropout(0.5)      
        )
        # third hidden layer
        self.hidden_layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(3, 3), padding=1), # size: 32, 256 layers
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)), # size: 16, 256 layers
            nn.Dropout(0.5)      
        )
        # fourth hidden layer
        self.hidden_layer4 = nn.Linear(256*16*16, 256) # size
        
        # output layer
        self.output_layer = nn.Linear(256, 3)
           
    # define forward propagation
    def forward(self, X):
        # return predict value using layers in constructor
        out = self.hidden_layer1(X)
        out = self.hidden_layer2(out)
        out = self.hidden_layer3(out)
        # transfer to 1-dim array
        out = out.view(out.shape[0], -1)
        out = self.hidden_layer4(out)
        out = self.output_layer(out)
        return out

In [41]:
# device settings, use graphic card if available
# Windows
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Mac ARM
# device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# Google Colab
device = torch.device('cuda:0')

model = CNN().to(device)

# set Cross Entropy as loss function
criterion = nn.CrossEntropyLoss()

# set Adam as optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [42]:
# define train function
def train(model, criterion, optimizer, loader):
  # define loss, acc variable for each epoch
  epoch_loss = 0
  epoch_acc = 0

  # set model training mode
  model.train()

  # start batch training
  for X_batch, y_batch in loader:
    # input data in batch to graphic card
    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
    # reset gradient
    optimizer.zero_grad()
    # train model
    hypothesis = model(X_batch)
    # calc loss
    loss = criterion(hypothesis, y_batch)
    # calc gradient
    loss.backward()
    # edit parameters using backpropagation
    optimizer.step()
    # calc acc
    y_predicted = torch.argmax(hypothesis, 1) # return max value's location (dim=1)
    acc = (y_predicted == y_batch).float().mean()
    # add loss, acc
    epoch_loss += loss.item()
    epoch_acc += acc.item()

  # return loss, acc of current epoch (mean value of each batch)
  return epoch_loss / len(loader), epoch_acc / len(loader)

In [43]:
# define evaluate function
def evaluate(model, criterion, loader):
  # define loss, acc variable for each epoch
  epoch_loss = 0
  epoch_acc = 0

  # set model evaluation mode
  model.eval()

  with torch.no_grad(): # to reduce memory when no grad
    # start batch testing
    for X_batch, y_batch in loader:
      # input data in batch to graphic card
      X_batch, y_batch = X_batch.to(device), y_batch.to(device)
      # test model
      hypothesis = model(X_batch)
      # calc loss
      loss = criterion(hypothesis, y_batch)
      # calc accuracy
      y_predicted = torch.argmax(hypothesis, 1)
      acc = (y_predicted == y_batch).float().mean()
      # add loss, acc
      epoch_loss += loss.item()
      epoch_acc += acc.item()

    # return loss, acc of current epoch (mean value of each batch)
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [44]:
# train model for 20 times
n_epochs = 20
for epoch in range(1, n_epochs+1):
  # train model
  loss, acc = train(model, criterion, optimizer, train_dataloader)
  # evaluate model
  test_loss, test_acc = evaluate(model, criterion, test_dataloader)

  # train result
  print('epoch: {}, loss: {:.3f}, acc: {:.2f}, test_loss: {:.3f}, test_acc: {:.3f}'
        .format(epoch, loss, acc, test_loss, test_acc))

epoch: 1, loss: 0.681, acc: 0.77, test_loss: 0.940, test_acc: 0.816
epoch: 2, loss: 0.507, acc: 0.78, test_loss: 0.761, test_acc: 0.821
epoch: 3, loss: 0.464, acc: 0.81, test_loss: 0.748, test_acc: 0.803
epoch: 4, loss: 0.445, acc: 0.83, test_loss: 0.665, test_acc: 0.847
epoch: 5, loss: 0.437, acc: 0.82, test_loss: 0.653, test_acc: 0.833
epoch: 6, loss: 0.422, acc: 0.84, test_loss: 0.578, test_acc: 0.848
epoch: 7, loss: 0.415, acc: 0.84, test_loss: 0.691, test_acc: 0.805
epoch: 8, loss: 0.407, acc: 0.84, test_loss: 0.670, test_acc: 0.837
epoch: 9, loss: 0.399, acc: 0.85, test_loss: 0.672, test_acc: 0.818
epoch: 10, loss: 0.391, acc: 0.85, test_loss: 0.620, test_acc: 0.838
epoch: 11, loss: 0.369, acc: 0.86, test_loss: 0.625, test_acc: 0.823
epoch: 12, loss: 0.363, acc: 0.86, test_loss: 0.561, test_acc: 0.852
epoch: 13, loss: 0.356, acc: 0.86, test_loss: 0.588, test_acc: 0.836
epoch: 14, loss: 0.337, acc: 0.87, test_loss: 0.492, test_acc: 0.872
epoch: 15, loss: 0.325, acc: 0.87, test_los

##5. Save Model for Inference

In [45]:
# print model info
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
hidden_layer1.0.weight 	 torch.Size([64, 1, 3, 3])
hidden_layer1.0.bias 	 torch.Size([64])
hidden_layer2.0.weight 	 torch.Size([128, 64, 3, 3])
hidden_layer2.0.bias 	 torch.Size([128])
hidden_layer3.0.weight 	 torch.Size([256, 128, 3, 3])
hidden_layer3.0.bias 	 torch.Size([256])
hidden_layer4.weight 	 torch.Size([256, 65536])
hidden_layer4.bias 	 torch.Size([256])
output_layer.weight 	 torch.Size([3, 256])
output_layer.bias 	 torch.Size([3])


In [46]:
# save model
save_path = '/content/drive/MyDrive/dataset/mask_model.pt'
torch.save(model.state_dict(), save_path)

##6. Load Model for Inference

In [47]:
# import model
load_path = '/content/drive/MyDrive/dataset/mask_model.pt'

class CNN(nn.Module):
    # define model structure
    def __init__(self):
        # call parent class(nn.Module)'s constructor
        super().__init__()
        
        # first hidden layer
        self.hidden_layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3), padding=1), # size: 128, 64 layers
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)), # size: 64, 64 layers
            nn.Dropout(0.5)
        )
        # second hidden layer
        self.hidden_layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3), padding=1), # size: 64, 128 layers
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)), # size: 32, 128 layers
            nn.Dropout(0.5)      
        )
        # third hidden layer
        self.hidden_layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(3, 3), padding=1), # size: 32, 256 layers
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)), # size: 16, 256 layers
            nn.Dropout(0.5)      
        )
        # fourth hidden layer
        self.hidden_layer4 = nn.Linear(256*16*16, 256) # size
        
        # output layer
        self.output_layer = nn.Linear(256, 3)
           
    # define forward propagation
    def forward(self, X):
        # return predict value using layers in constructor
        out = self.hidden_layer1(X)
        out = self.hidden_layer2(out)
        out = self.hidden_layer3(out)
        # transfer to 1-dim array
        out = out.view(out.shape[0], -1)
        out = self.hidden_layer4(out)
        out = self.output_layer(out)
        return out

device = torch.device('cuda:0')

model = CNN().to(device)
model.load_state_dict(torch.load(load_path))
model.eval()

CNN(
  (hidden_layer1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.5, inplace=False)
  )
  (hidden_layer2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.5, inplace=False)
  )
  (hidden_layer3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.5, inplace=False)
  )
  (hidden_layer4): Linear(in_features=65536, out_features=256, bias=True)
  (output_layer): Linear(in_features=256, out_features=3, bias=True)
)

In [48]:
# print model info
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
hidden_layer1.0.weight 	 torch.Size([64, 1, 3, 3])
hidden_layer1.0.bias 	 torch.Size([64])
hidden_layer2.0.weight 	 torch.Size([128, 64, 3, 3])
hidden_layer2.0.bias 	 torch.Size([128])
hidden_layer3.0.weight 	 torch.Size([256, 128, 3, 3])
hidden_layer3.0.bias 	 torch.Size([256])
hidden_layer4.weight 	 torch.Size([256, 65536])
hidden_layer4.bias 	 torch.Size([256])
output_layer.weight 	 torch.Size([3, 256])
output_layer.bias 	 torch.Size([3])
